In [82]:
from jours_feries_france import JoursFeries

def get_jour_ferie(date):
    # Récupérer tous les jours fériés de l'année 2025
    jours_feries = JoursFeries.for_year(date.year)

    # Vérifier si la date du jour est dans la liste des jours fériés
    est_ferie = False
    for nom, date_ferie in jours_feries.items():
        if date_ferie == date:
            est_ferie = True
            break

    if est_ferie:
        print("Aujourd'hui est un jour férié en France.")
    else:
        print("Aujourd'hui n'est pas un jour férié en France.")
    

    return est_ferie

In [83]:
from datetime import datetime

# Obtenir la date du jour
aujourdhui = datetime.today()

print(aujourdhui.year)
print(aujourdhui.month)
print(aujourdhui.day)

# Weekday + 1 car en France la semaine commence lundi alors que datetime utilise les weekday anglosaxon et leur semaine commence le dimanche
print('Numéro du jour de la semaine : ', aujourdhui.isoweekday())

get_jour_ferie(aujourdhui)

2025
7
30
Numéro du jour de la semaine :  3
Aujourd'hui n'est pas un jour férié en France.


False

In [15]:
import requests

def lecture_api_scenario(url_base_api_injecteur, identifiant_scenario):
    """
    Charge les données du scénario depuis l'API si la lecture est activée
    et si un identifiant est présent dans la configuration.
    """
    url = f"{url_base_api_injecteur}injapi/scenario/{identifiant_scenario}"

    response = requests.get(url, timeout=10)
    # Raises :class:`HTTPError`, if one occurred."""
    response.raise_for_status()

    return response.json()


donnees_api = lecture_api_scenario('http://192.168.210.205/', '1f0095e0-d201-6b36-a2d0-391ef0b9d7cb')

i = 1

for key in donnees_api.keys():
    print(i, key, donnees_api[key])
    i += 1

1 id 287082
2 nom aai2_consultation_demande_v6
3 identifiant 1f0095e0-d201-6b36-a2d0-391ef0b9d7cb
4 date_creation 2025-03-25 10:48:02
5 ancien_nom None
6 description None
7 id_application 15
8 id_planning_execution 1
9 id_planning_indispo 1
10 id_planning_maintenance None
11 pas_execution 1200
12 seuil_message 40
13 seuil_alerte None
14 flag_affichage None
15 flag_maintenance None
16 motif_maintenance None
17 flag_ferie None
18 flag_actif True
19 date_activation 2025-06-27 16:44:32
20 date_desactivation 2025-06-27 16:44:18
21 flag_indispo None
22 flag_export None
23 indispo_ferie None
24 date_activ_dispo None
25 date_fin_periode None
26 screenshots None
27 archive False
28 updated_at 2025-07-30 16:03:59
29 date_dernier_status None
30 id_synchro 20
31 seuil_ralentissement None
32 planning [{'jour': 1, 'heure_debut': '07:00:00', 'heure_fin': '22:00:00'}, {'jour': 2, 'heure_debut': '07:00:00', 'heure_fin': '22:00:00'}, {'jour': 3, 'heure_debut': '07:00:00', 'heure_fin': '22:00:00'}, {'jou

In [95]:
# Variable dont on a besoin pour vérifier si le scénarios est dans la bonne période d'execution
# flag_ferie, planning

from datetime import datetime

def check_flag_ferie(flag_ferie):
    """
    Vérifier si le scénario à l'autorisation de s'executer en cas de jours férié 
    """

    # Si on est un jour férié et que le flag férié est différent de True il faut annuler le lancement du scénario
    # flag_ferie peut être None, True ou False
    if flag_ferie is not True:
        # Renvoyer une erreur et arrêter pytest
        pass
    else:
        # On peut continuer le traitement
        pass

def get_plages_horaires(planning):
    """
    Création d'une liste des différentes plages horaires sur lesquelles le scénarios doit s'executer sur la journée
    """
    plages_horaires = []

    # Si le planning est dans les données API on recherche les différentes plages horaires
    if planning:
        for plage_horaire in planning:
            if plage_horaire['jour'] == aujourdhui.isoweekday():
                print(plage_horaire)
                plages_horaires.append(plage_horaire)
    
    return plages_horaires

def check_planning_execution(plages_horaires, date_du_jour):
    """
    Vérifier si le scénario à l'autorisation de s'executer.
    """

    for plage_horaire in plages_horaires:
        heure_debut = datetime.strptime(plage_horaire['heure_debut'], '%H:%M:%S').time()
        heure_fin = datetime.strptime(plage_horaire['heure_fin'], '%H:%M:%S').time()
        print(heure_debut, type(heure_debut))
        print(date_du_jour.time(), type(date_du_jour.time()))
        print(heure_fin, type(heure_fin))

        # Si le scénario est dans une plage horaire il peut s'executer sinon il faut vérifier la prochaine plage horaire de la liste.
        # Si le scénario n'a pas l'autorisation de s'executer dans au moins l'une des plages horaires alors il faut l'arrêter avant son lancement

def planning_execution():
    """
    Vérifie si le planning d'exécution est bien respecté
    """
    # Obtenir la date du jour
    aujourdhui = datetime.today()

    # Récupération du flag férié dans les données
    flag_ferie = donnees_api['flag_ferie']

    # Récupération du planning de la journée
    planning = donnees_api.get('planning', None)

    # Si on est férié on vérifier si le scénario doit tourner ou non
    if get_jour_ferie(aujourdhui) is True:
        check_flag_ferie(date_du_jour=aujourdhui, flag_ferie=flag_ferie)

    # Création de la liste des différentes plages horaires sur lesquelles le scénarios doit être actif sur la journée
    plages_horaires = get_plages_horaires(planning)

    # Vérifier si l'exection se déroule dans l'une des plages horaires
    check_planning_execution(plages_horaires=plages_horaires, date_du_jour=aujourdhui)

planning_execution()

Aujourd'hui n'est pas un jour férié en France.
{'jour': 3, 'heure_debut': '07:00:00', 'heure_fin': '22:00:00'}
07:00:00 <class 'datetime.time'>
17:47:17.324391 <class 'datetime.time'>
22:00:00 <class 'datetime.time'>
Dedans
